In [1]:
import pandas as pd
import numpy as np
import ast

In [49]:
top_5_df = pd.read_csv('Data/top_5_df.csv')
top_5_df.rename(columns={'Unnamed: 0':'id'}, inplace=True)
top_5_df.reset_index(inplace=True)
top_5_df.drop('index', axis=1, inplace=True)
top_5_df.set_index(['id','name'], inplace=True)
#print(top_5_df.iloc[1][1])
top_5_df.shape

(72230, 2)

In [3]:
indices = pd.Series(top_5_df.index.get_level_values(1))
#indices

In [4]:
#TRY TO GET DROP DOWN LIST WITH ARTIST NAME FOR SONGS WITH SAME NAME
def num_of_artists(name):
    artists = []
    name = name.lower()
    for i in range(len(indices[indices==name])):
        idx = indices[indices == name].index[i]
        artists.append(top_5_df.iloc[idx][0])
    return artists

In [17]:
def recommendations(name, artists):
    #INPUT A SONG RETURN
    name = name.lower()
    ind = 0
    for i in range(len(indices[indices == name])):
        if(top_5_df.iloc[indices[indices == name].index[i]][0] == artists):
            ind = i
    idx = indices[indices == name].index[ind]
    #print(top_5_df.iloc[idx][0])
    
    top = top_5_df.iloc[idx]
    top_5_dict = ast.literal_eval(top.top_5)
    return top_5_dict

In [18]:
#snow = recommendations('one time', 'Justin Bieber')
#for key, value in snow.items():
 #   print(key, value[0])

In [19]:
import warnings
warnings.filterwarnings('ignore')

import ipywidgets as widgets
from IPython.core.display import display, HTML
from IPython.display import display, clear_output

In [20]:
#!jupyter nbextension enable --py widgetsnbextension --sys-prefix
#!jupyter serverextension enable --sys-prefix voila

In [21]:
song_name = widgets.Text(placeholder='Please enter a song')

In [22]:
artists = num_of_artists(song_name.value)

In [23]:
artist = widgets.Dropdown(
            options=artists,
            description='Artists Name',
            disbaled=False)

In [24]:
def update_song(change):
    song_name.value = change['new']
    artists = num_of_artists(song_name.value)
    artist.options = artists
song_name.observe(update_song, 'value')


In [45]:
get_top_5 = widgets.Button(
                description='Find recommended songs',
                tooltip='Find Song',
                layout=widgets.Layout(width='30%'),
                style={'description_width': 'intitial'}
            )
output = widgets.Output(layout=widgets.Layout(width='100%'))

def on_button_clicked(event):
    with output:
        clear_output()
        try:
            top_5 = recommendations(str(song_name.value), str(artist.value))
            display(HTML(f'<h3>Here are the top 5 recommended songs for {song_name.value} by {artist.value}</h3>'))
            for key, value in top_5.items():
                url = f'https://open.spotify.com/track/{key}'
                display(HTML('<a href={}>{},{}</a>'.format(url, value[1], value[0])))
        except IndexError:
            display(HTML(f'<h3>Sorry Song Not Found! Please Enter in another Song</h3>'))
        
get_top_5.on_click(on_button_clicked)

vbox_result = widgets.VBox([get_top_5, output])

In [46]:
hbox_result = widgets.HBox([song_name, artist])

In [47]:
text_0 = widgets.HTML(value='<h1>Jacob\'s Jukebox<h1>', layout=widgets.Layout(align_content='center'))
text_1 = widgets.HTML(value='<h2>Enter in a song to find the top 5 recommended songs!')

vbox_text = widgets.VBox([text_0, text_1, hbox_result, vbox_result], layout=widgets.Layout(align_content='center'))

In [48]:
page = widgets.HBox([vbox_text], layout=widgets.Layout(align_content= 'center'))
display(page)